CLEAN DATA FOR QUANTITATIVE

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.9 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import re
import pandas as pd
import emoji

DATA_PATH   = Path("DataPost.csv")
OUTPUT_PATH = Path("data_hijabistahub_clean.csv")

def load_apify_dump(path: Path) -> pd.DataFrame:
    if path.suffix.lower() == ".csv":
        return pd.read_csv(path)
    if path.suffix.lower() in (".json", ".jsonl"):
        return pd.read_json(path, orient="records", lines=True)
    raise ValueError(f"Unsupported file type: {path.suffix}")

df = load_apify_dump(DATA_PATH)

#Drop low-utility / internal columns
drop_cols = ["input","effectStickers/0/ID", "effectStickers/0/name",
       "effectStickers/0/stickerStats/useCount", "effectStickers/1/ID",
       "effectStickers/1/name", "effectStickers/1/stickerStats/useCount",
       "effectStickers/2/ID", "effectStickers/2/name",
       "effectStickers/2/stickerStats/useCount", "effectStickers/3/ID",
       "effectStickers/3/name", "effectStickers/3/stickerStats/useCount",
       "effectStickers/4/ID", "effectStickers/4/name",
       "effectStickers/4/stickerStats/useCount"]
df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)

#flatten column names  (/ and \ → _)
df.columns = df.columns.str.replace(r"[\\/]", "_", regex=True)

#numeric engagement metrics
numeric_cols = [
    "collectCount",
    "commentCount",
    "diggCount",
    "playCount",
    "shareCount",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = (
            df[col]
              .astype(str)
              .str.replace(",", "", regex=False)      # "1,234" → "1234"
              .replace(r"^\s*$", pd.NA, regex=True)   # blank → NA
              .astype(float)
        )


#Boolean flags
bool_cols = ["isAd", "isMuted", "isPinned", "isSlideshow", "isSponsored"]
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype("boolean")


#Datetime parsing
if "createTimeISO" in df.columns:
    df["post_datetime"] = pd.to_datetime(df["createTimeISO"], errors="coerce")
df.drop(columns=[c for c in ("createTime", "createTimeISO") if c in df.columns],
        inplace=True)


#Collapse hashtag columns into one string
hashtag_cols = [c for c in df.columns if re.fullmatch(r"hashtags_\d+_name", c)]
if hashtag_cols:
    df["hashtags"] = (
        df[hashtag_cols]
          .apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)
    )
    df.drop(columns=hashtag_cols, inplace=True)


#Collapse mention columns into one string
mention_cols = [c for c in df.columns if re.fullmatch(r"mentions_\d+", c)]
if mention_cols:
    df["mentions"] = (
        df[mention_cols]
          .apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)
    )
    df.drop(columns=mention_cols, inplace=True)

def clean_caption_all(text):
    if pd.isna(text):
        return ""

    text = text.lower()                             # lowercase
    text = re.sub(r"\.{2,}", ".", text)             # collapse dots
    text = re.sub(r"\s+", " ", text).strip()        # normalize spaces
    text = emoji.demojize(text)                     # 😍 → :heart_eyes:
    text = re.sub(r"http\S+", "", text)             # remove URLs
    text = re.sub(r"@\w+", "", text)                # remove @mentions
    text = re.sub(r"[^\w\s:]", "", text)            # keep colon for emoji text
    text = re.sub(r"\s+", " ", text).strip()        # remove extra whitespace
    return text

df['cleaned_text'] = df['text'].apply(clean_caption_all)

#Save clean CSV
df.to_csv(OUTPUT_PATH, index=False)
print(f"✅  Clean file written to {OUTPUT_PATH}  "
      f"({df.shape[0]:,d} rows × {df.shape[1]} columns)")


✅  Clean file written to data_hijabistahub_clean.csv  (447 rows × 27 columns)


In [ ]:
import pandas as pd
df = pd.read_csv("DataComment.csv")
print(df.columns)

Index(['avatarThumbnail', 'cid', 'createTime', 'createTimeISO',
       'detailedMentions/0/nickName', 'detailedMentions/0/profileUrl',
       'detailedMentions/0/secUid', 'detailedMentions/0/userId',
       'detailedMentions/1/nickName', 'detailedMentions/1/profileUrl',
       'detailedMentions/1/secUid', 'detailedMentions/1/userId',
       'detailedMentions/2/nickName', 'detailedMentions/2/profileUrl',
       'detailedMentions/2/secUid', 'detailedMentions/2/userId', 'diggCount',
       'input', 'likedByAuthor', 'mentions/0', 'mentions/1', 'mentions/2',
       'pinnedByAuthor', 'repliesToId', 'replyCommentTotal',
       'submittedVideoUrl', 'text', 'uid', 'uniqueId', 'videoWebUrl'],
      dtype='object')


SPLIT DATASET

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------------
# 1. Load the cleaned comment data
#    (replace with your actual path / filename)
# ------------------------------------------------------------------
DATA_PATH = "HijabistahubComment_labelled.csv"
df = pd.read_csv(DATA_PATH)

# ------------------------------------------------------------------
# 2. Split into train (80 %) and un-labelled test (20 %)
#    • If you ALREADY have a 'sentiment' column, drop it before splitting
#      and join it back to the train set only.
#    • random_state fixes the shuffle order so the split is reproducible.
# ------------------------------------------------------------------
train_df, test_df = train_test_split(
    df,
    test_size=0.20,
    random_state=42,
    shuffle=True
)

# ------------------------------------------------------------------
# 3. Make sure the test set really has no sentiment label
#    (Here we just add an empty column as a placeholder)
# ------------------------------------------------------------------
test_df["sentiment"] = pd.NA          # or test_df.drop(columns=['sentiment'], inplace=True)

# ------------------------------------------------------------------
# 4. Save both files
# ------------------------------------------------------------------
train_df.to_csv("comments_train_to_label.csv", index=False)
test_df.to_csv("comments_test_unlabelled.csv", index=False)

print(f"✅ Train set: {len(train_df):,} rows → comments_train_to_label.csv")
print(f"✅ Un-labelled test set: {len(test_df):,} rows → comments_test_unlabelled.csv")


✅ Train set: 713 rows → comments_train_to_label.csv
✅ Un-labelled test set: 179 rows → comments_test_unlabelled.csv


CLEAN DATA FOR COMMENTS (QUANLITATIVE)

In [ ]:
!pip install malaya
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.3 MB/s eta 0:00:00


In [ ]:
import html, unicodedata
import re, emoji, pandas as pd, malaya
import pandas as pd

/usr/local/lib/python3.11/dist-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/usr/local/lib/python3.11/dist-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [ ]:
# Load your dataset
df = pd.read_csv('DataComment.csv')

# Load your comment-level dataset
df = pd.read_csv("DataComment.csv")

# Flatten column names (replace / with _)
df.columns = df.columns.str.replace(r"[\\/]", "_", regex=True)

# Drop unnecessary columns
drop_cols = [c for c in df.columns if any(key in c for key in [
    "avatarThumbnail", "cid", "uid", "secUid", "profileUrl", "repliesToId", "input"
])]
df.drop(columns=drop_cols, inplace=True)

# Collapse mentions
mention_cols = [c for c in df.columns if c.startswith("mentions_")]
if mention_cols:
    df["mentions"] = df[mention_cols].apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)
    df.drop(columns=mention_cols, inplace=True)

# Collapse mentions
mention_cols = [c for c in df.columns if c.startswith("detailedMentions_")]
if mention_cols:
    df["detailedMentions"] = df[mention_cols].apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)
    df.drop(columns=mention_cols, inplace=True)

#convert datetime
df["comment_datetime"] = pd.to_datetime(df["createTimeISO"], errors="coerce")
df.drop(columns=["createTime", "createTimeISO"], inplace=True)

# Load normalizer
normalizer = malaya.normalize.normalizer()

# Remove weird unicode
def remove_weird_unicode(text):
    text = text.replace('²', '2')
    text = text.replace('¹', '1')
    return text

def unicode_html_normalize(text: str) -> str:
    if not isinstance(text, str):
        return ''
    text = html.unescape(text)
    text = unicodedata.normalize('NFKC', text)
    text = text.replace('\u200b', '')
    return text

URL_PATTERN    = re.compile(r'https?://\S+|www\.\S+')
EMAIL_PATTERN  = re.compile(r'\b\S+@\S+\.\S+\b')

def mask_urls_emails(text: str) -> str:
    if not isinstance(text, str):
        return ''
    text = URL_PATTERN.sub('[URL]', text)
    text = EMAIL_PATTERN.sub('[EMAIL]', text)
    return text

#remove any mention
def remove_user_mentions(text):
    if not isinstance(text, str):
        return text
    # Remove any @username pattern (alphanumeric, underscores, dots)
    return re.sub(r'@\w+', '', text)

def fix_apostrophes(text):
    return text.replace("’", "'")

contractions = {
    "it's": "it is",
    "isn't": "is not",
    "don't": "do not",
    "can't": "cannot",
    "i'm": "i am",
    "you're": "you are",
    "they're": "they are",
    "we're": "we are",
    "didn't": "did not",
    "YOURE" : "you are"
}

def expand_contractions(text):
    for contraction, full in contractions.items():
        text = text.replace(contraction, full)
    return text

# replace Malay shortforms
shortform_mapping = {
    'yg': 'yang', 'tk': 'tak', 'x': 'tak', 'xde': 'tiada',
    'takde': 'tak ada', 'tak de': 'tak ada', 'nk': 'nak',
    'gi': 'pergi', 'dgn': 'dengan', 'blh': 'boleh',
    'tp': 'tapi', 'ko': 'kau', 'lg': 'lagi', 'org': 'orang',
    'depa': 'mereka', 'tu': 'itu', 'sbb': 'sebab', 'msti': 'mesti',
    'ni': 'ini', 'bang': 'abang'
}

def replace_shortforms(text):
    if not isinstance(text, str):
        return text
    for short, full in shortform_mapping.items():
        pattern = r'\b' + re.escape(short) + r'\b'  # Match whole word
        text = re.sub(pattern, full, text)
    return text

#convert emojis to text
def convert_emoji_to_text(text):
    return emoji.demojize(text)

#fix multiple punctuations
def fix_multiple_punctuations(text):
    text = re.sub(r'\s*\.\s*', '.', text)  # fix dots
    text = re.sub(r'\s*!+\s*', '!', text)   # fix exclamations
    text = re.sub(r'\s*\?+\s*', '?', text)  # fix questions
    text = re.sub(r'\.{2,}', '.', text)  # multiple dots → one dot
    text = re.sub(r'!{2,}', '!', text)   # multiple ! → one !
    text = re.sub(r'\?{2,}', '?', text)  # multiple ? → one ?
    return text

#expands double word
def expand_double_words(text):
    if not isinstance(text, str):
        return text
    # Only expand if "ii" is a standalone word (with spaces or punctuation)
    text = re.sub(r'\b(\w+)\s+i{2}\b', r'\1 \1', text)  # handles "sama ii"
    return text

#Apply full cleaning pipeline
def full_cleaning_pipeline(text):
    if not isinstance(text, str):
        return ''
    text = unicode_html_normalize(text)
    text = mask_urls_emails(text)
    text = remove_user_mentions(text)
    text = remove_weird_unicode(text)
    text = fix_apostrophes(text)
    text = expand_contractions(text.lower())
    text = replace_shortforms(text)
    text = convert_emoji_to_text(text)
    text = expand_double_words(text)
    # Collapse stray whitespace that may have been introduced
    text = re.sub(r'\s{2,}', ' ', text).strip()
    return text

# Apply to create 'cleaned_text' column
df['cleaned_text'] = df['text'].apply(full_cleaning_pipeline)

# Normalize using Malaya
def normalize_text(text):
    if not isinstance(text, str):
        return ''
    try:
        normalized = normalizer.normalize(text)
        if isinstance(normalized, dict):
            return normalized['normalize']
        else:
            return normalized
    except:
        return text

df['cleaned_text'] = df['cleaned_text'].apply(normalize_text)

# Fix multiple punctuation and lowercase
df['cleaned_text'] = df['cleaned_text'].apply(fix_multiple_punctuations)
df['cleaned_text'] = df['cleaned_text'].str.lower()

#custom mapping that are done by manually check one by one the cleaned data if there
#is any mislook shortforms or mispell words that malaya couldnt catch
custom_mapping = {'abis':'habis','skali':'sekali','aja':'saja','siutlaa':'siotla','datok':'dato',
                  'collaberation':'collaboration','kawin':'kahwin','kawen':'kahwin','kawhinn':'kahwin',
                  'fakehopa':'fake hope','western':'western','diorng':'diorang','lpsnie':'lepasni',
                  'sorg':'sorang','dorang':'diorang','mmg':'memang','sy':'saya','dlu':'dulu','lak':'pulak',
                  'ku':'aku','tak':'tidak','nmpk':'nampak','mls':'malas','ambk':'ambil','datu-datu':'dato dato',
                  'pebenda':'apa benda','kne':'kena','ak':'aku','trik':'tarik','bapak':'bapa','mals':'malas',
                  'bpk':'bapa','gak':'juga','mcm':'macam','maleh':'malas','die':'dia','blkng':'belakang',
                  'jnji':'janji','lsg':'langsung','ktwa':'ketawa','ptut':'patut','sye':'saya','giler':'gila',
                  'tuh':'tu','cantek':'cantik','okey':'okay','ensemm':'hensem','sgt':'sangat','bru':'baru',
                  'bals':'balas','cam':'macam','btl-btl':'betul-betul','stu':'satu','klu':'kalau','nnti':'nanti',
                  'tgok':'tengok','datuk':'dato','ikam':'ikan','zizang':'zizan','tggu':'tunggu','jer':'je',
                  'punchlina':'punchline','byk':'banyak','msa':'masa','dlu':'dulu','ne':'ni','ntah':'entah',
                  'hbis':'habis','knpela':'kenapala','drpd':'daripada','laki':'lelaki','dye':'dia','pulokk':'pula',
                  'suprisa':'suprise','get back':'getback','niy':'ni','dr':'dari','bnr':'benar','zan':'zizan',
                  'ambik':'ambil','cntik':'cantik','logat':'loghat','bgus':'bagus','lurva':'lurve','sesunguh':'sesungguhnya',
                  'adek':'adik','cutta':'cute','aq':'aku','tgk':'tengok','spa':'siapa','lpsni':'lepasni','mlysia':'malaysia',
                  'gile':'gila','dieorang':'diorang','pkai':'pakai','klo':'kalau','internasional':'international',
                  'gilers':'gila','klatei':'kelantan','dio':'dia','kelate':'kelantan','taglina':'tagline','comey':'comel',
                  'mampuihh':'mampus','mmpos':'mampus','teruseesss':'terus','trus':'terus','prmpuan':'perempuan','gilos':'gila',
                  'adekk':'adik','lupe':'lupa','dorg':'diorang','canteknya':'cantiknya','coleb':'collab','ade':'ada',
                  'sunaa':'cuna','jy':'juga','kunen':'kuning','gak':'tak','tknk':'taknak','dorang':'diorang','alip':'alif',
                  'besaqq':'besar','canti':'cantik','daviena':'davina','ko':'kau','ari':'hari','sgt':'sangat',
                  'hamble':'humble','sppu':'sepupu','comeyy':'comel','syantiknyaa':'cantiknya','micel':'michelle',
                  'viba':'vibe','mekap':'makeup','pikap line':'pickupline','klate':'kelantan',
                  'kiteorang':'kitaorang','hay':'hai','btl':'betul','jugak':'juga','vers':'versi','kawinn':'kahwin',
                  'gewa':'gewe','klo':'kalau'
                  }


# Function to replace only full words using regex
def replace_custom_shortforms(text, mapping):
    if not isinstance(text, str):
        return text
    for short, full in mapping.items():
        pattern = r'\b' + re.escape(short) + r'\b'
        text = re.sub(pattern, full, text)
    return text

# Apply the function directly to cleaned_text
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: replace_custom_shortforms(x, custom_mapping))



df.to_csv("DataCommentTesting.csv", index=False)
